## Import Libraries

In [184]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import re

## Import Dataset

In [185]:
# Baca file TSV
df = pd.read_csv('/content/uniprotkb_AND_reviewed_true_AND_existen_2025_04_22.tsv', sep='\t')

# Tampilkan 5 baris pertama untuk memeriksa data
df.head(5)

,Entry,Entry Name,Protein names,Gene Names,Organism,Length,Sequence,Reviewed,Helix,Turn,Beta strand,PubMed ID,DOI ID
0,A0A068B6Q6,CA18_CONBE,Conotoxin Bt1.8,NaN,Conus betulinus (Beech cone),37,PDGRNAAAKAFDLITPTVRKGCCSNPACILNNPNQCG,reviewed,"HELIX 28..31; /evidence=""ECO:0007829|PDB:2NAY""",NaN,"STRAND 23..26; /evidence=""ECO:0007829|PDB:2NAY""",34008858,10.1111/jnc.15434
1,A0A088MIT0,BRKP2_PHYNA,Bradykinin-related peptides [Cleaved into: Nat...,BBN,Physalaemus nattereri (Cuyaba dwarf frog) (Eup...,134,MAFLKKSLFLVLFLGVVSLSFCEEEKREEHEEEKRDEEDAESLGKR...,reviewed,NaN,NaN,NaN,26443407,10.1002/rcm.7313
2,A0A097PTA8,DEFCO_COPCI,Fungal defensin copsin,NaN,Coprinopsis cinerea (Inky cap fungus) (Hormogr...,184,MKLSTSLLAIVAVASTFIGNALSATTVPGCFAECIDKAAVAVNCAA...,reviewed,NaN,NaN,NaN,25342741; 28825809,10.1074/jbc.m114.599878; 10.1021/acs.biochem.7...
3,A0A0A0LLY1,SRBP1_CUCSA,Small RNA binding protein 1 (CsSRBP1),SRBP1 Csa_2G247040,Cucumis sativus (Cucumber),160,MASSSVEFRCFVGGLAWATDSNSLEKAFSVYGEIVEAKIVSDRETG...,reviewed,NaN,NaN,NaN,31812689; 19881527,10.1016/j.molp.2019.12.001; 10.1038/ng.475
4,A0A0A1I6E7,NDB4S_ANDCR,Antimicrobial peptide AcrAP1,NaN,Androctonus crassicauda (Arabian fat-tailed sc...,74,MEIKYLLTVFLVLLIVSDHCQAFLFSLIPHAISGLISAFKGRRKRD...,reviewed,NaN,NaN,NaN,25332684,10.7150/ijbs.9859


In [186]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32761 entries, 0 to 32760
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Entry          32761 non-null  object
 1   Entry Name     32761 non-null  object
 2   Protein names  32761 non-null  object
 3   Gene Names     21071 non-null  object
 4   Organism       32761 non-null  object
 5   Length         32761 non-null  int64 
 6   Sequence       32761 non-null  object
 7   Reviewed       32761 non-null  object
 8   Helix          7642 non-null   object
 9   Turn           5059 non-null   object
 10  Beta strand    6887 non-null   object
 11  PubMed ID      31352 non-null  object
 12  DOI ID         31222 non-null  object
dtypes: int64(1), object(12)
memory usage: 3.2+ MB


In [187]:
unique_values = df.nunique()
print("Unique values in each column:")
print(unique_values)

Unique values in each column:
Entry            32761
Entry Name       32761
Protein names    24023
Gene Names       18007
Organism          4413
Length             199
Sequence         30518
Reviewed             1
Helix             7638
Turn              5055
Beta strand       6884
PubMed ID        21368
DOI ID           21207
dtype: int64


In [188]:
df = df[['Sequence','Length', 'Organism', 'Helix', 'Turn', 'Beta strand']]
df

,Sequence,Length,Organism,Helix,Turn,Beta strand
0,PDGRNAAAKAFDLITPTVRKGCCSNPACILNNPNQCG,37,Conus betulinus (Beech cone),"HELIX 28..31; /evidence=""ECO:0007829|PDB:2NAY""",NaN,"STRAND 23..26; /evidence=""ECO:0007829|PDB:2NAY"""
1,MAFLKKSLFLVLFLGVVSLSFCEEEKREEHEEEKRDEEDAESLGKR...,134,Physalaemus nattereri (Cuyaba dwarf frog) (Eup...,NaN,NaN,NaN
2,MKLSTSLLAIVAVASTFIGNALSATTVPGCFAECIDKAAVAVNCAA...,184,Coprinopsis cinerea (Inky cap fungus) (Hormogr...,NaN,NaN,NaN
3,MASSSVEFRCFVGGLAWATDSNSLEKAFSVYGEIVEAKIVSDRETG...,160,Cucumis sativus (Cucumber),NaN,NaN,NaN
4,MEIKYLLTVFLVLLIVSDHCQAFLFSLIPHAISGLISAFKGRRKRD...,74,Androctonus crassicauda (Arabian fat-tailed sc...,NaN,NaN,NaN
...,...,...,...,...,...,...
32756,MLEKVYVALIHYPIKGKDGSIISTAVTNLDVHDIARTARTYNLKGY...,192,Thermotoga maritima (strain ATCC 43589 / DSM 3...,"HELIX 28..40; /evidence=""ECO:0007829|PDB:3DCM""...","TURN 161..164; /evidence=""ECO:0007829|PDB:3DCM...","STRAND 4..10; /evidence=""ECO:0007829|PDB:3DCM""..."
32757,MTTSRRPSPRIRSFVKDLSATIPGAFRFTRGHYSMEELAREAIIRG...,180,Aeropyrum pernix (strain ATCC 700893 / DSM 118...,"HELIX 9..19; /evidence=""ECO:0007829|PDB:2CXH"";...",NaN,"STRAND 1..6; /evidence=""ECO:0007829|PDB:2CXH"";..."
32758,MAAKTKTLELEDNVFLLLEGNLKRIFATPIGYTTFREFQNVVFNCA...,184,Chlamydia pneumoniae (Chlamydophila pneumoniae),"HELIX 12..25; /evidence=""ECO:0007829|PDB:3Q9D""...",NaN,"STRAND 100..111; /evidence=""ECO:0007829|PDB:3Q..."
32759,MAISAEELEKILKKSFPSSVIKITDLVGDQDHYALEISDAQFNGLS...,77,Rickettsia prowazekii (strain Madrid E),"HELIX 5..12; /evidence=""ECO:0007829|PDB:2MCQ"";...","TURN 13..15; /evidence=""ECO:0007829|PDB:2MCQ"";...","STRAND 17..19; /evidence=""ECO:0007829|PDB:2MCQ..."


## Feature Engineering

In [189]:
# Fungsi untuk mengekstrak angka dari string dalam format "X..Y"
def extract_ranges(text):
    """Ekstrak semua range angka dalam format "X..Y" dari teks"""
    if pd.isna(text):
        return []
    pattern = r'(\d+)\.\.(\d+)'
    matches = re.findall(pattern, text)
    # Konversi setiap pasangan angka menjadi integer
    ranges = [(int(start), int(end)) for start, end in matches]
    return ranges

# Fungsi untuk menghitung jumlah struktur dalam setiap kolom
def count_structures(df, column_name):
    """Menghitung jumlah struktur dalam setiap baris untuk kolom tertentu"""
    if column_name not in df.columns:
        return 0

    counts = []
    for idx, row in df.iterrows():
        if pd.isna(row[column_name]):
            counts.append(0)
        else:
            structures = extract_ranges(row[column_name])
            counts.append(len(structures))

    return counts

# Hitung jumlah struktur untuk setiap baris
df['helix_count'] = count_structures(df, 'Helix')
df['turn_count'] = count_structures(df, 'Turn')
df['strand_count'] = count_structures(df, 'Beta strand')

# Hitung nilai maksimum dari jumlah struktur
max_helix = df['helix_count'].max() if 'helix_count' in df.columns else 0
max_turn = df['turn_count'].max() if 'turn_count' in df.columns else 0
max_strand = df['strand_count'].max() if 'strand_count' in df.columns else 0

print("Jumlah maksimum dari struktur:")
print(f"Maksimum Helix: {max_helix}")
print(f"Maksimum Turn: {max_turn}")
print(f"Maksimum Strand: {max_strand}")

# Fungsi untuk membuat kolom individual untuk setiap pasangan start-end
def create_structure_columns(df, column_name, prefix):
    """
    Membuat kolom individual untuk setiap struktur, misalnya:
    HELIX_1_start, HELIX_1_end, HELIX_2_start, HELIX_2_end, dst.
    """
    # Temukan jumlah maksimum struktur di semua baris
    max_structures = 0
    for idx, row in df.iterrows():
        if pd.isna(row[column_name]):
            continue
        structures = extract_ranges(row[column_name])
        max_structures = max(max_structures, len(structures))

    print(f"Jumlah maksimum struktur {prefix}: {max_structures}")

    # Buat kolom kosong dengan nilai NaN untuk semua struktur
    for i in range(1, max_structures + 1):
        df[f'{prefix}_{i}_start'] = np.nan
        df[f'{prefix}_{i}_end'] = np.nan

    # Isi kolom dengan nilai dari setiap baris
    for idx, row in df.iterrows():
        if pd.isna(row[column_name]):
            continue
        structures = extract_ranges(row[column_name])
        for i, (start, end) in enumerate(structures, 1):
            if i <= max_structures:  # Pastikan tidak melebihi jumlah kolom yang dibuat
                df.at[idx, f'{prefix}_{i}_start'] = start
                df.at[idx, f'{prefix}_{i}_end'] = end

    return df, max_structures

# Terapkan fungsi ke setiap kolom struktur dan dapatkan jumlah maksimum struktur
df, max_helix_cols = create_structure_columns(df, 'Helix', 'HELIX')
df, max_turn_cols = create_structure_columns(df, 'Turn', 'TURN')
df, max_strand_cols = create_structure_columns(df, 'Beta strand', 'STRAND')

# Tampilkan hasil dengan kolom baru - Helix
helix_cols = [col for col in df.columns if 'HELIX_' in col]
turn_cols = [col for col in df.columns if 'TURN_' in col]
strand_cols = [col for col in df.columns if 'STRAND_' in col]

# Hitung jumlah struktur untuk setiap baris
print("\nJumlah struktur per baris:")
print(df[['Sequence', 'helix_count', 'turn_count', 'strand_count']])

# Menampilkan statistik jumlah struktur
print("\nStatistik jumlah struktur:")
print("Helix:")
print(f"  Minimum: {df['helix_count'].min()}")
print(f"  Maksimum: {df['helix_count'].max()}")
print(f"  Rata-rata: {df['helix_count'].mean():.2f}")
print("Turn:")
print(f"  Minimum: {df['turn_count'].min()}")
print(f"  Maksimum: {df['turn_count'].max()}")
print(f"  Rata-rata: {df['turn_count'].mean():.2f}")
print("Strand:")
print(f"  Minimum: {df['strand_count'].min()}")
print(f"  Maksimum: {df['strand_count'].max()}")
print(f"  Rata-rata: {df['strand_count'].mean():.2f}")

# Contoh cara menggunakan data dari kolom individual
print("\nNilai dari kolom struktur untuk baris pertama:")
print(f"HELIX_1_start: {df['HELIX_1_start'][0]}, HELIX_1_end: {df['HELIX_1_end'][0]}")
if max_turn_cols >= 1:
    print(f"TURN_1_start: {df['TURN_1_start'][0]}, TURN_1_end: {df['TURN_1_end'][0]}")
if max_strand_cols >= 1:
    print(f"STRAND_1_start: {df['STRAND_1_start'][0]}, STRAND_1_end: {df['STRAND_1_end'][0]}")

# Menampilkan semua kolom untuk melihat struktur lengkap dataframe
print("\nSemua kolom dalam dataframe:")
print(df.columns.tolist())

# Cara menggunakan informasi jumlah maksimum struktur
print("\nJumlah maksimum struktur (dari fungsi):")
print(f"Maksimum kolom Helix: {max_helix_cols}")
print(f"Maksimum kolom Turn: {max_turn_cols}")
print(f"Maksimum kolom Strand: {max_strand_cols}")

Jumlah maksimum dari struktur:
Maksimum Helix: 17
Maksimum Turn: 7
Maksimum Strand: 22
Jumlah maksimum struktur HELIX: 17
Jumlah maksimum struktur TURN: 7
Jumlah maksimum struktur STRAND: 22

Jumlah struktur per baris:
                                                Sequence  helix_count  \
0                  PDGRNAAAKAFDLITPTVRKGCCSNPACILNNPNQCG            1   
1      MAFLKKSLFLVLFLGVVSLSFCEEEKREEHEEEKRDEEDAESLGKR...            0   
2      MKLSTSLLAIVAVASTFIGNALSATTVPGCFAECIDKAAVAVNCAA...            0   
3      MASSSVEFRCFVGGLAWATDSNSLEKAFSVYGEIVEAKIVSDRETG...            0   
4      MEIKYLLTVFLVLLIVSDHCQAFLFSLIPHAISGLISAFKGRRKRD...            0   
...                                                  ...          ...   
32756  MLEKVYVALIHYPIKGKDGSIISTAVTNLDVHDIARTARTYNLKGY...            8   
32757  MTTSRRPSPRIRSFVKDLSATIPGAFRFTRGHYSMEELAREAIIRG...            4   
32758  MAAKTKTLELEDNVFLLLEGNLKRIFATPIGYTTFREFQNVVFNCA...            7   
32759  MAISAEELEKILKKSFPSSVIKITDLVGDQDHYALEISDAQFNG

In [190]:
import pandas as pd
import numpy as np

# Anggap df adalah dataset Anda
# df = pd.read_csv('your_dataset.csv')

# Fungsi umum untuk mengekstrak sequence berdasarkan indeks start dan end
def extract_sequence(row, start_col, end_col):
    # Periksa apakah start dan end bukan NaN
    if pd.notna(row[start_col]) and pd.notna(row[end_col]):
        # Convert ke integer dan kurangi 1 jika indeks dimulai dari 0
        # (tergantung pada apakah indeks dalam dataset dimulai dari 0 atau 1)
        start_idx = int(row[start_col]) - 1  # kurangi 1 jika indeks dimulai dari 0
        end_idx = int(row[end_col])  # tidak dikurangi jika end_idx inklusif

        # Ekstrak sequence berdasarkan indeks
        if pd.notna(row['Sequence']):
            return row['Sequence'][start_idx:end_idx]
    return np.nan

# 1. Membuat kolom HELIX_n
helix_cols = [col for col in df.columns if col.startswith('HELIX_') and '_start' in col]
max_helix = len(helix_cols)

for i in range(1, max_helix + 1):
    start_col = f'HELIX_{i}_start'
    end_col = f'HELIX_{i}_end'
    new_col = f'HELIX_{i}'

    if start_col in df.columns and end_col in df.columns:
        df[new_col] = df.apply(lambda row: extract_sequence(row, start_col, end_col), axis=1)

# 2. Membuat kolom TURN_n
turn_cols = [col for col in df.columns if col.startswith('TURN_') and '_start' in col]
max_turn = len(turn_cols)

for i in range(1, max_turn + 1):
    start_col = f'TURN_{i}_start'
    end_col = f'TURN_{i}_end'
    new_col = f'TURN_{i}'

    if start_col in df.columns and end_col in df.columns:
        df[new_col] = df.apply(lambda row: extract_sequence(row, start_col, end_col), axis=1)

# 3. Membuat kolom STRAND_n
strand_cols = [col for col in df.columns if col.startswith('STRAND_') and '_start' in col]
max_strand = len(strand_cols)

for i in range(1, max_strand + 1):
    start_col = f'STRAND_{i}_start'
    end_col = f'STRAND_{i}_end'
    new_col = f'STRAND_{i}'

    if start_col in df.columns and end_col in df.columns:
        df[new_col] = df.apply(lambda row: extract_sequence(row, start_col, end_col), axis=1)

# Menampilkan contoh hasilnya
print(df.head())

<ipython-input-190-023035dc7b6f>:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col] = df.apply(lambda row: extract_sequence(row, start_col, end_col), axis=1)
<ipython-input-190-023035dc7b6f>:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col] = df.apply(lambda row: extract_sequence(row, start_col, end_col), axis=1)
<ipython-input-190-023035dc7b6f>:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joi

                                            Sequence  Length  \
0              PDGRNAAAKAFDLITPTVRKGCCSNPACILNNPNQCG      37   
1  MAFLKKSLFLVLFLGVVSLSFCEEEKREEHEEEKRDEEDAESLGKR...     134   
2  MKLSTSLLAIVAVASTFIGNALSATTVPGCFAECIDKAAVAVNCAA...     184   
3  MASSSVEFRCFVGGLAWATDSNSLEKAFSVYGEIVEAKIVSDRETG...     160   
4  MEIKYLLTVFLVLLIVSDHCQAFLFSLIPHAISGLISAFKGRRKRD...      74   

                                            Organism  \
0                       Conus betulinus (Beech cone)   
1  Physalaemus nattereri (Cuyaba dwarf frog) (Eup...   
2  Coprinopsis cinerea (Inky cap fungus) (Hormogr...   
3                         Cucumis sativus (Cucumber)   
4  Androctonus crassicauda (Arabian fat-tailed sc...   

                                            Helix Turn  \
0  HELIX 28..31; /evidence="ECO:0007829|PDB:2NAY"  NaN   
1                                             NaN  NaN   
2                                             NaN  NaN   
3                                             

<ipython-input-190-023035dc7b6f>:55: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col] = df.apply(lambda row: extract_sequence(row, start_col, end_col), axis=1)


In [191]:
import pandas as pd

# Mengatur agar semua kolom ditampilkan
pd.set_option('display.max_columns', None)

# Menampilkan 5 baris pertama dengan semua kolom
display(df.head())


,Sequence,Length,Organism,Helix,Turn,Beta strand,helix_count,turn_count,strand_count,HELIX_1_start,HELIX_1_end,HELIX_2_start,HELIX_2_end,HELIX_3_start,HELIX_3_end,HELIX_4_start,HELIX_4_end,HELIX_5_start,HELIX_5_end,HELIX_6_start,HELIX_6_end,HELIX_7_start,HELIX_7_end,HELIX_8_start,HELIX_8_end,HELIX_9_start,HELIX_9_end,HELIX_10_start,HELIX_10_end,HELIX_11_start,HELIX_11_end,HELIX_12_start,HELIX_12_end,HELIX_13_start,HELIX_13_end,HELIX_14_start,HELIX_14_end,HELIX_15_start,HELIX_15_end,HELIX_16_start,HELIX_16_end,HELIX_17_start,HELIX_17_end,TURN_1_start,TURN_1_end,TURN_2_start,TURN_2_end,TURN_3_start,TURN_3_end,TURN_4_start,TURN_4_end,TURN_5_start,TURN_5_end,TURN_6_start,TURN_6_end,TURN_7_start,TURN_7_end,STRAND_1_start,STRAND_1_end,STRAND_2_start,STRAND_2_end,STRAND_3_start,STRAND_3_end,STRAND_4_start,STRAND_4_end,STRAND_5_start,STRAND_5_end,STRAND_6_start,STRAND_6_end,STRAND_7_start,STRAND_7_end,STRAND_8_start,STRAND_8_end,STRAND_9_start,STRAND_9_end,STRAND_10_start,STRAND_10_end,STRAND_11_start,STRAND_11_end,STRAND_12_start,STRAND_12_end,STRAND_13_start,STRAND_13_end,STRAND_14_start,STRAND_14_end,STRAND_15_start,STRAND_15_end,STRAND_16_start,STRAND_16_end,STRAND_17_start,STRAND_17_end,STRAND_18_start,STRAND_18_end,STRAND_19_start,STRAND_19_end,STRAND_20_start,STRAND_20_end,STRAND_21_start,STRAND_21_end,STRAND_22_start,STRAND_22_end,HELIX_1,HELIX_2,HELIX_3,HELIX_4,HELIX_5,HELIX_6,HELIX_7,HELIX_8,HELIX_9,HELIX_10,HELIX_11,HELIX_12,HELIX_13,HELIX_14,HELIX_15,HELIX_16,HELIX_17,TURN_1,TURN_2,TURN_3,TURN_4,TURN_5,TURN_6,TURN_7,STRAND_1,STRAND_2,STRAND_3,STRAND_4,STRAND_5,STRAND_6,STRAND_7,STRAND_8,STRAND_9,STRAND_10,STRAND_11,STRAND_12,STRAND_13,STRAND_14,STRAND_15,STRAND_16,STRAND_17,STRAND_18,STRAND_19,STRAND_20,STRAND_21,STRAND_22
0,PDGRNAAAKAFDLITPTVRKGCCSNPACILNNPNQCG,37,Conus betulinus (Beech cone),"HELIX 28..31; /evidence=""ECO:0007829|PDB:2NAY""",NaN,"STRAND 23..26; /evidence=""ECO:0007829|PDB:2NAY""",1,0,1,28.0,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CILN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CSNP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MAFLKKSLFLVLFLGVVSLSFCEEEKREEHEEEKRDEEDAESLGKR...,134,Physalaemus nattereri (Cuyaba dwarf frog) (Eup...,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MKLSTSLLAIVAVASTFIGNALSATTVPGCFAECIDKAAVAVNCAA...,184,Coprinopsis cinerea (Inky cap fungus) (Hormogr...,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MASSSVEFRCFVGGLAWATDSNSLEKAFSVYGEIVEAKIVSDRETG...,160,Cucumis sativus (Cucumber),NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [192]:
# prompt: tampilkan df dengan length = 5 AA dan helix atau strand atau turn tidak null

# Filter the DataFrame
filtered_df = df[(df['Length'] == 7) &
                 ((df['Helix'].notnull()) | (df['Turn'].notnull()) | (df['Beta strand'].notnull()))]

# Display the filtered DataFrame
display(filtered_df)


,Sequence,Length,Organism,Helix,Turn,Beta strand,helix_count,turn_count,strand_count,HELIX_1_start,HELIX_1_end,HELIX_2_start,HELIX_2_end,HELIX_3_start,HELIX_3_end,HELIX_4_start,HELIX_4_end,HELIX_5_start,HELIX_5_end,HELIX_6_start,HELIX_6_end,HELIX_7_start,HELIX_7_end,HELIX_8_start,HELIX_8_end,HELIX_9_start,HELIX_9_end,HELIX_10_start,HELIX_10_end,HELIX_11_start,HELIX_11_end,HELIX_12_start,HELIX_12_end,HELIX_13_start,HELIX_13_end,HELIX_14_start,HELIX_14_end,HELIX_15_start,HELIX_15_end,HELIX_16_start,HELIX_16_end,HELIX_17_start,HELIX_17_end,TURN_1_start,TURN_1_end,TURN_2_start,TURN_2_end,TURN_3_start,TURN_3_end,TURN_4_start,TURN_4_end,TURN_5_start,TURN_5_end,TURN_6_start,TURN_6_end,TURN_7_start,TURN_7_end,STRAND_1_start,STRAND_1_end,STRAND_2_start,STRAND_2_end,STRAND_3_start,STRAND_3_end,STRAND_4_start,STRAND_4_end,STRAND_5_start,STRAND_5_end,STRAND_6_start,STRAND_6_end,STRAND_7_start,STRAND_7_end,STRAND_8_start,STRAND_8_end,STRAND_9_start,STRAND_9_end,STRAND_10_start,STRAND_10_end,STRAND_11_start,STRAND_11_end,STRAND_12_start,STRAND_12_end,STRAND_13_start,STRAND_13_end,STRAND_14_start,STRAND_14_end,STRAND_15_start,STRAND_15_end,STRAND_16_start,STRAND_16_end,STRAND_17_start,STRAND_17_end,STRAND_18_start,STRAND_18_end,STRAND_19_start,STRAND_19_end,STRAND_20_start,STRAND_20_end,STRAND_21_start,STRAND_21_end,STRAND_22_start,STRAND_22_end,HELIX_1,HELIX_2,HELIX_3,HELIX_4,HELIX_5,HELIX_6,HELIX_7,HELIX_8,HELIX_9,HELIX_10,HELIX_11,HELIX_12,HELIX_13,HELIX_14,HELIX_15,HELIX_16,HELIX_17,TURN_1,TURN_2,TURN_3,TURN_4,TURN_5,TURN_6,TURN_7,STRAND_1,STRAND_2,STRAND_3,STRAND_4,STRAND_5,STRAND_6,STRAND_7,STRAND_8,STRAND_9,STRAND_10,STRAND_11,STRAND_12,STRAND_13,STRAND_14,STRAND_15,STRAND_16,STRAND_17,STRAND_18,STRAND_19,STRAND_20,STRAND_21,STRAND_22
20956,MRTGNAN,7,Escherichia coli,NaN,NaN,"STRAND 3..5; /evidence=""ECO:0007829|PDB:3H9G""",0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TGN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31629,LVTLVFV,7,Enterococcus faecalis (Streptococcus faecalis),NaN,NaN,"STRAND 2..4; /evidence=""ECO:0007829|PDB:4FAJ""",0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VTL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [193]:
# prompt: hapus kolom  'helix_count', 'turn_count', 'strand_count', 'HELIX_1_start', 'HELIX_1_end', 'HELIX_2_start', 'HELIX_2_end', 'HELIX_3_start', 'HELIX_3_end', 'HELIX_4_start', 'HELIX_4_end', 'HELIX_5_start', 'HELIX_5_end', 'HELIX_6_start', 'HELIX_6_end', 'HELIX_7_start', 'HELIX_7_end', 'HELIX_8_start', 'HELIX_8_end', 'HELIX_9_start', 'HELIX_9_end', 'HELIX_10_start', 'HELIX_10_end', 'HELIX_11_start', 'HELIX_11_end', 'HELIX_12_start', 'HELIX_12_end', 'HELIX_13_start', 'HELIX_13_end', 'HELIX_14_start', 'HELIX_14_end', 'HELIX_15_start', 'HELIX_15_end', 'HELIX_16_start', 'HELIX_16_end', 'HELIX_17_start', 'HELIX_17_end', 'TURN_1_start', 'TURN_1_end', 'TURN_2_start', 'TURN_2_end', 'TURN_3_start', 'TURN_3_end', 'TURN_4_start', 'TURN_4_end', 'TURN_5_start', 'TURN_5_end', 'TURN_6_start', 'TURN_6_end', 'TURN_7_start', 'TURN_7_end', 'STRAND_1_start', 'STRAND_1_end', 'STRAND_2_start', 'STRAND_2_end', 'STRAND_3_start', 'STRAND_3_end', 'STRAND_4_start', 'STRAND_4_end', 'STRAND_5_start', 'STRAND_5_end', 'STRAND_6_start', 'STRAND_6_end', 'STRAND_7_start', 'STRAND_7_end', 'STRAND_8_start', 'STRAND_8_end', 'STRAND_9_start', 'STRAND_9_end', 'STRAND_10_start', 'STRAND_10_end', 'STRAND_11_start', 'STRAND_11_end', 'STRAND_12_start', 'STRAND_12_end', 'STRAND_13_start', 'STRAND_13_end', 'STRAND_14_start', 'STRAND_14_end', 'STRAND_15_start', 'STRAND_15_end', 'STRAND_16_start', 'STRAND_16_end', 'STRAND_17_start', 'STRAND_17_end', 'STRAND_18_start', 'STRAND_18_end', 'STRAND_19_start', 'STRAND_19_end', 'STRAND_20_start', 'STRAND_20_end', 'STRAND_21_start', 'STRAND_21_end', 'STRAND_22_start', 'STRAND_22_end'

cols_to_drop = ['Helix', 'Turn', 'Beta strand', 'HELIX_1_start', 'HELIX_1_end', 'HELIX_2_start', 'HELIX_2_end', 'HELIX_3_start', 'HELIX_3_end', 'HELIX_4_start', 'HELIX_4_end', 'HELIX_5_start', 'HELIX_5_end', 'HELIX_6_start', 'HELIX_6_end', 'HELIX_7_start', 'HELIX_7_end', 'HELIX_8_start', 'HELIX_8_end', 'HELIX_9_start', 'HELIX_9_end', 'HELIX_10_start', 'HELIX_10_end', 'HELIX_11_start', 'HELIX_11_end', 'HELIX_12_start', 'HELIX_12_end', 'HELIX_13_start', 'HELIX_13_end', 'HELIX_14_start', 'HELIX_14_end', 'HELIX_15_start', 'HELIX_15_end', 'HELIX_16_start', 'HELIX_16_end', 'HELIX_17_start', 'HELIX_17_end', 'TURN_1_start', 'TURN_1_end', 'TURN_2_start', 'TURN_2_end', 'TURN_3_start', 'TURN_3_end', 'TURN_4_start', 'TURN_4_end', 'TURN_5_start', 'TURN_5_end', 'TURN_6_start', 'TURN_6_end', 'TURN_7_start', 'TURN_7_end', 'STRAND_1_start', 'STRAND_1_end', 'STRAND_2_start', 'STRAND_2_end', 'STRAND_3_start', 'STRAND_3_end', 'STRAND_4_start', 'STRAND_4_end', 'STRAND_5_start', 'STRAND_5_end', 'STRAND_6_start', 'STRAND_6_end', 'STRAND_7_start', 'STRAND_7_end', 'STRAND_8_start', 'STRAND_8_end', 'STRAND_9_start', 'STRAND_9_end', 'STRAND_10_start', 'STRAND_10_end', 'STRAND_11_start', 'STRAND_11_end', 'STRAND_12_start', 'STRAND_12_end', 'STRAND_13_start', 'STRAND_13_end', 'STRAND_14_start', 'STRAND_14_end', 'STRAND_15_start', 'STRAND_15_end', 'STRAND_16_start', 'STRAND_16_end', 'STRAND_17_start', 'STRAND_17_end', 'STRAND_18_start', 'STRAND_18_end', 'STRAND_19_start', 'STRAND_19_end', 'STRAND_20_start', 'STRAND_20_end', 'STRAND_21_start', 'STRAND_21_end', 'STRAND_22_start', 'STRAND_22_end']
df = df.drop(columns=cols_to_drop, errors='ignore')


In [194]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32761 entries, 0 to 32760
Data columns (total 52 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Sequence      32761 non-null  object
 1   Length        32761 non-null  int64 
 2   Organism      32761 non-null  object
 3   helix_count   32761 non-null  int64 
 4   turn_count    32761 non-null  int64 
 5   strand_count  32761 non-null  int64 
 6   HELIX_1       7642 non-null   object
 7   HELIX_2       6547 non-null   object
 8   HELIX_3       5495 non-null   object
 9   HELIX_4       4389 non-null   object
 10  HELIX_5       3385 non-null   object
 11  HELIX_6       2494 non-null   object
 12  HELIX_7       1740 non-null   object
 13  HELIX_8       1152 non-null   object
 14  HELIX_9       745 non-null    object
 15  HELIX_10      427 non-null    object
 16  HELIX_11      210 non-null    object
 17  HELIX_12      98 non-null     object
 18  HELIX_13      42 non-null     object
 19  HELI

In [195]:
import pandas as pd
import numpy as np

# Misalkan df adalah dataframe asli Anda
# df = pd.read_csv('your_data.csv')  # Ganti dengan kode untuk membaca data Anda

# Membuat dataframe kosong untuk hasil akhir
result_df = pd.DataFrame(columns=['length_seq', 'Organism', 'subsequence', 'length_sub_seq', 'type'])

# Ambil kolom utama yang akan disertakan di semua baris
lengths = df['Length'].values
organisms = df['Organism'].values
sequences = df['Sequence'].values  # Jika diperlukan

# Fungsi untuk memproses kolom dan menambahkan ke dataframe hasil
def process_columns(df, prefix, type_name):
    rows_list = []

    # Ambil jumlah entri dalam dataframe asli
    n_entries = len(df)

    # Cari kolom yang cocok dengan prefix (HELIX_, TURN_, atau STRAND_)
    cols = [col for col in df.columns if col.startswith(prefix)]

    # Untuk setiap entri dalam dataframe asli
    for i in range(n_entries):
        length_seq = df.loc[i, 'Length']
        organism = df.loc[i, 'Organism']

        # Untuk setiap kolom yang cocok dengan prefix
        for col in cols:
            # Ambil nilai sub-sequence, lewati jika NaN
            sub_seq = df.loc[i, col]
            if pd.notna(sub_seq):
                # Hitung panjang sub-sequence
                length_sub_seq = len(str(sub_seq))

                rows_list.append({
                    'length_seq': length_seq,
                    'Organism': organism,
                    'subsequence': sub_seq,
                    'length_sub_seq': length_sub_seq,
                    'type': type_name
                })

    return pd.DataFrame(rows_list)

# Proses setiap tipe struktur
helix_df = process_columns(df, 'HELIX_', 'helix')
turn_df = process_columns(df, 'TURN_', 'turn')
strand_df = process_columns(df, 'STRAND_', 'strand')

# Gabungkan semua hasil
result_df = pd.concat([helix_df, turn_df, strand_df], ignore_index=True)

# Tampilkan informasi tentang dataframe hasil
print(result_df.info())
print("\nContoh beberapa baris pertama:")
print(result_df.head())

# Jika ingin menyimpan hasil ke file
# result_df.to_csv('transformed_data.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81989 entries, 0 to 81988
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   length_seq      81989 non-null  int64 
 1   Organism        81989 non-null  object
 2   subsequence     81989 non-null  object
 3   length_sub_seq  81989 non-null  int64 
 4   type            81989 non-null  object
dtypes: int64(2), object(3)
memory usage: 3.1+ MB
None

Contoh beberapa baris pertama:
   length_seq                                       Organism  \
0          37                   Conus betulinus (Beech cone)   
1         147  Aphanizomenon flos-aquae (strain 2012/KM1/D3)   
2         147  Aphanizomenon flos-aquae (strain 2012/KM1/D3)   
3         147  Aphanizomenon flos-aquae (strain 2012/KM1/D3)   
4         147  Aphanizomenon flos-aquae (strain 2012/KM1/D3)   

                     subsequence  length_sub_seq   type  
0                           CILN               4  heli

In [196]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Load the DataFrame (assuming df is already loaded)

def perform_anova(df):
    # Check if the required columns are in the DataFrame
    if 'length_seq' not in df.columns or 'type' not in df.columns:
        print("DataFrame must contain 'length_seq' and 'type' columns.")
        return

    try:
        # Convert the 'type' column to a categorical type if it's not
        df['type'] = df['type'].astype('category')

        # ANOVA using OLS
        model = ols('length_seq ~ C(type)', data=df).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)

        print("ANOVA Results:")
        print(anova_table)
        return anova_table

    except Exception as e:
        print("Error while performing ANOVA:", str(e))

# Example usage
anova_result = perform_anova(result_df)


ANOVA Results:
                sum_sq       df          F        PR(>F)
C(type)   1.995775e+05      2.0  67.115624  7.514326e-30
Residual  1.218983e+08  81986.0        NaN           NaN


In [197]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Load the DataFrame (assuming df is already loaded)

def perform_anova(df):
    # Check if the required columns are in the DataFrame
    if 'length_sub_seq' not in df.columns or 'type' not in df.columns:
        print("DataFrame must contain 'length_sub_seq' and 'type' columns.")
        return

    try:
        # Convert the 'type' column to a categorical type if it's not
        df['type'] = df['type'].astype('category')

        # ANOVA using OLS
        model = ols('length_sub_seq ~ C(type)', data=df).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)

        print("ANOVA Results:")
        print(anova_table)
        return anova_table

    except Exception as e:
        print("Error while performing ANOVA:", str(e))

# Example usage
anova_result = perform_anova(result_df)


ANOVA Results:
                sum_sq       df            F  PR(>F)
C(type)   4.140723e+05      2.0  8218.524463     0.0
Residual  2.065342e+06  81986.0          NaN     NaN


In [198]:
# prompt: drop length_seq di result_df

# Drop the specified columns
cols_to_drop = ['length_seq']
result_df = result_df.drop(columns=cols_to_drop, errors='ignore')


In [199]:
import pandas as pd
from scipy.stats import chi2_contingency

# Generate a contingency table between organism and type
contingency_table = pd.crosstab(result_df['Organism'], result_df['type'])

# Perform the Chi-Square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

print(pd.DataFrame(expected, index=contingency_table.index, columns=contingency_table.columns))

# Interpret the result
alpha = 0.05
if p < alpha:
    print("Reject the null hypothesis: There is a significant association between organism and type.")
else:
    print("Fail to reject the null hypothesis: No significant association between organism and type.")


type                                                    helix     strand  \
Organism                                                                   
Acanthamoeba castellanii (Amoeba)                   20.551684  23.079108   
Acanthamoeba polyphaga mimivirus (APMV)              5.452488   6.123029   
Acanthoscurria gomesiana (Tarantula spider) (Ph...   0.838844   0.942004   
Acaryochloris marina (strain MBIC 11017)             3.774799   4.239020   
Acetivibrio thermocellus (strain ATCC 27405 / D...  18.873995  21.195099   
...                                                       ...        ...   
Zea mays subsp. parviglumis (Balsas teosinte)        3.355377   3.768018   
Zoarces americanus (Ocean pout) (Macrozoarces a...   6.291332   7.065033   
Zymomonas mobilis subsp. mobilis (strain ATCC 1...   5.871910   6.594031   
Zymomonas mobilis subsp. mobilis (strain ATCC 3...   4.194221   4.710022   
Zymoseptoria tritici (strain CBS 115943 / IPO32...   2.516533   2.826013   

type       

In [200]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81989 entries, 0 to 81988
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Organism        81989 non-null  object  
 1   subsequence     81989 non-null  object  
 2   length_sub_seq  81989 non-null  int64   
 3   type            81989 non-null  category
dtypes: category(1), int64(1), object(2)
memory usage: 2.0+ MB


In [201]:
# prompt: drop length_sub_seq yang lebih dari 7

# Filter out rows where 'length_sub_seq' is greater than 7
filtered_result_df = result_df[result_df['length_sub_seq'] <= 7].copy()

# Display the info of the filtered DataFrame
print(filtered_result_df.info())

# Optionally display the first few rows of the filtered DataFrame
print("\nFirst few rows of the filtered DataFrame (length_sub_seq <= 7):")
print(filtered_result_df.head())

# You can assign this filtered DataFrame back to result_df if you want to replace the original
# result_df = filtered_result_df


<class 'pandas.core.frame.DataFrame'>
Index: 52831 entries, 0 to 81988
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Organism        52831 non-null  object  
 1   subsequence     52831 non-null  object  
 2   length_sub_seq  52831 non-null  int64   
 3   type            52831 non-null  category
dtypes: category(1), int64(1), object(2)
memory usage: 1.7+ MB
None

First few rows of the filtered DataFrame (length_sub_seq <= 7):
                                            Organism subsequence  \
0                       Conus betulinus (Beech cone)        CILN   
2      Aphanizomenon flos-aquae (strain 2012/KM1/D3)         KKF   
3      Aphanizomenon flos-aquae (strain 2012/KM1/D3)       LDDYL   
6      Aphanizomenon flos-aquae (strain 2012/KM1/D3)     PELAKQI   
9  Conus geographus (Geography cone) (Nubecula ge...       VVEHC   

   length_sub_seq   type  
0               4  helix  
2               3  helix 

In [202]:
# prompt: hitung frekuensi type

# Hitung frekuensi kemunculan setiap 'type'
type_counts = result_df['type'].value_counts()

print("Frekuensi kemunculan setiap tipe struktur:")
type_counts


Frekuensi kemunculan setiap tipe struktur:


,count
type,
strand,38617
helix,34388
turn,8984


In [203]:
# Drop duplicate rows from result_df
result_df = result_df.drop_duplicates()


## Data Standarization

In [204]:
# Create a StandardScaler object
scaler = StandardScaler()

# Fit the scaler on the numerical features of the training set
# Assuming 'length_sub_seq' is the only numerical feature
scaler.fit(result_df[['length_sub_seq']])

# Transform the numerical features of the validation set using the same scaler
result_df['length_sub_seq'] = scaler.transform(result_df[['length_sub_seq']])

## Data

In [205]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Membuat objek LabelEncoder
label_encoder = LabelEncoder()

# Melakukan label encoding pada kolom 'type'
result_df['type_Encoded'] = label_encoder.fit_transform(result_df['type'])

In [206]:
result_df = result_df.drop(columns=['type'], errors='ignore')


In [207]:
result_df

,Organism,subsequence,length_sub_seq,type_Encoded
0,Conus betulinus (Beech cone),CILN,-0.616144,0
1,Aphanizomenon flos-aquae (strain 2012/KM1/D3),EPVIIETRLELIGRYLDHL,2.094104,0
2,Aphanizomenon flos-aquae (strain 2012/KM1/D3),KKF,-0.796827,0
3,Aphanizomenon flos-aquae (strain 2012/KM1/D3),LDDYL,-0.435461,0
4,Aphanizomenon flos-aquae (strain 2012/KM1/D3),FEQQLITERLLQLITQAAIDINDHILSKL,3.900936,0
...,...,...,...,...
81984,Chlamydia pneumoniae (Chlamydophila pneumoniae),EFLLMTD,-0.074094,1
81985,Rickettsia prowazekii (strain Madrid E),PSS,-0.796827,1
81986,Rickettsia prowazekii (strain Madrid E),KITDLVGDQD,0.467955,1
81987,Rickettsia prowazekii (strain Madrid E),YALEISDAQFNG,0.829322,1


In [208]:
# prompt: ubah nilai Organism yang memiliki frekuensi < 11  menjadi 'Other'

# Ubah nilai 'Organism' yang memiliki frekuensi < 11 menjadi 'Other'
low_frequency_organisms = organism_counts[organism_counts < 11].index
result_df['Organism'] = result_df['Organism'].apply(lambda x: 'Other' if x in low_frequency_organisms else x)

# Periksa hasilnya dengan melihat frekuensi 'Organism' setelah perubahan
print("\nFrekuensi Organism setelah mengubah frekuensi < 11 menjadi 'Other':")
print(result_df['Organism'].value_counts())


Frekuensi Organism setelah mengubah frekuensi < 11 menjadi 'Other':
Organism
Homo sapiens (Human)                                                            10258
Escherichia coli (strain K12)                                                    5547
Saccharomyces cerevisiae (strain ATCC 204508 / S288c) (Baker's yeast)            3625
Mus musculus (Mouse)                                                             3398
Other                                                                            3114
                                                                                ...  
Thunnus orientalis (North Pacific bluefin tuna) (Thunnus thynnus orientalis)       11
Escherichia coli O6:K15:H31 (strain 536 / UPEC)                                    11
Hemachatus haemachatus (Rinkhals) (Sepedon haemachatus)                            11
Naja pallida (Red spitting cobra)                                                  11
Plantago lanceolata (English plantain) (Ribwort plantain)     

In [209]:
# prompt: cetak result_df yang memiliki Organism other

result_df[result_df['Organism'] == 'Other']


,Organism,subsequence,length_sub_seq,type_Encoded
0,Other,CILN,-0.616144,0
17,Other,EELDAFHQIFTTTKEAIERFMAMLTPVIE,3.900936,0
18,Other,DHERLYYHHIYEEEEQRLSRLDVLIPLIEKFQDE,4.804352,0
19,Other,NNAFNRLLQELNLEKFGLHNFIEHVDLALFS,4.262302,0
20,Other,EERQTLLKELRKDAYEGYQYVKEKLAE,3.539569,0
...,...,...,...,...
81833,Other,EFKIKFKDPLKVT,1.010005,1
81834,Other,DSSNSV,-0.254778,1
81835,Other,ITIDQYTYD,0.287272,1
81836,Other,DDD,-0.796827,1


In [210]:
# prompt: ubah nilai subsequence yang memiliki frekuensi < 2  menjadi 'Other'

# Ubah nilai 'subsequence' yang memiliki frekuensi < 2 menjadi 'Other'
low_frequency_subsequences = subsequence_counts[subsequence_counts < 2].index
result_df['subsequence'] = result_df['subsequence'].apply(lambda x: 'Other' if x in low_frequency_subsequences else x)

# Periksa hasilnya dengan melihat frekuensi 'subsequence' setelah perubahan
print("\nFrekuensi subsequence setelah mengubah frekuensi < 2 menjadi 'Other':")
print(result_df['subsequence'].value_counts())


Frekuensi subsequence setelah mengubah frekuensi < 2 menjadi 'Other':
subsequence
Other          51646
PEL               39
ADK               33
PDG               31
PEE               30
               ...  
PPG                2
SRQAQDLARSY        2
RKYN               2
AGFK               2
MGE                2
Name: count, Length: 7600, dtype: int64


In [211]:
# prompt: buatlah target encode dari Organism dengan target type

# Calculate target encoding for 'Organism' based on 'type_Encoded'
organism_target_encoding = result_df.groupby('Organism')['type_Encoded'].mean()

# Map the target encoding to the 'Organism' column in the DataFrame
result_df['Organism'] = result_df['Organism'].map(organism_target_encoding)

# prompt: buatlah target encode dari subsequence dengan target type

# Calculate target encoding for 'subsequence' based on 'type_Encoded'
subsequence_target_encoding = result_df.groupby('subsequence')['type_Encoded'].mean()

# Map the target encoding to the 'subsequence' column in the DataFrame
result_df['subsequence'] = result_df['subsequence'].map(subsequence_target_encoding)

# Display the updated DataFrame
result_df


,Organism,subsequence,length_sub_seq,type_Encoded
0,0.666988,0.585738,-0.616144,0
1,0.320000,0.585738,2.094104,0
2,0.320000,0.333333,-0.796827,0
3,0.320000,0.585738,-0.435461,0
4,0.320000,0.585738,3.900936,0
...,...,...,...,...
81984,0.500000,0.585738,-0.074094,1
81985,0.898305,0.818182,-0.796827,1
81986,0.898305,0.585738,0.467955,1
81987,0.898305,0.585738,0.829322,1


In [212]:
# prompt: buatlah undersample dari type_encoded dari result_df

import pandas as pd
from imblearn.under_sampling import RandomUnderSampler

# Assuming 'type_Encoded' is your target variable
X = result_df.drop('type_Encoded', axis=1)
y = result_df['type_Encoded']

# Initialize RandomUnderSampler
rus = RandomUnderSampler(random_state=42)  # Set random_state for reproducibility

# Resample the data
X_resampled, y_resampled = rus.fit_resample(X, y)

# Create a new DataFrame with the resampled data
undersampled_df = pd.DataFrame(X_resampled, columns=X.columns)
undersampled_df['type_Encoded'] = y_resampled

# Now 'undersampled_df' contains your undersampled data
undersampled_df


,Organism,subsequence,length_sub_seq,type_Encoded
32506,0.700000,0.585738,0.287272,0
15527,0.729488,0.585738,1.010005,0
24660,0.666988,0.585738,0.648638,0
11598,0.703474,0.585738,-0.616144,0
6208,0.685124,0.585738,0.106589,0
...,...,...,...,...
43367,0.703474,1.500000,-0.616144,2
43368,0.653901,0.585738,-0.616144,2
43369,0.653901,0.833333,-0.796827,2
43370,0.898305,1.111111,-0.796827,2


In [213]:
# prompt: buatlah model cnn untuk undersampled_df

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
import pandas as pd # Ensure pandas is imported if not already

# Assuming 'undersampled_df' is your DataFrame and 'length_sub_seq', 'Organism',
# and the binary encoded 'subsequence' columns are your features
# and 'type_Encoded' is your target variable.

# Prepare the data
# Get the list of binary encoded subsequence columns
subsequence_binary_cols = [col for col in undersampled_df.columns if col.startswith('subsequence_binary_')]

# Combine the feature columns
feature_cols = ['length_sub_seq', 'Organism'] + subsequence_binary_cols

X = undersampled_df[feature_cols].values
y = undersampled_df['type_Encoded'].values

# Reshape X for 1D CNN (add a channel dimension)
# The second dimension is now the number of features (length_sub_seq, Organism, and all binary columns)
X = X.reshape(X.shape[0], X.shape[1], 1)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the CNN model
model = Sequential()
# The input_shape should match the shape of your training data (number of features, 1)
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1), padding='same'))
model.add(MaxPooling1D(pool_size=2))  # Adjust pool_size if needed, but with padding='same' it might be less critical
model.add(Flatten())
model.add(Dense(32, activation='relu'))  # Adjust units as needed
# model.add(Dropout(0.2))  # Add dropout for regularization
model.add(Dense(128, activation='relu'))  # Adjust units as needed
# model.add(Dropout(0.2))  # Add dropout for regularization
model.add(Dense(3, activation='softmax'))  # Output layer with softmax for multi-class classification


# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=8, validation_split=0.1)  # Adjust epochs and batch_size as needed

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2313/2313 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.5786 - loss: 0.8947 - val_accuracy: 0.6055 - val_loss: 0.8627
Epoch 2/10
2313/2313 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.6126 - loss: 0.8516 - val_accuracy: 0.6138 - val_loss: 0.8387
Epoch 3/10
2313/2313 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.6063 - loss: 0.8533 - val_accuracy: 0.6119 - val_loss: 0.8396
Epoch 4/10
2313/2313 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.6016 - loss: 0.8582 - val_accuracy: 0.6119 - val_loss: 0.8366
Epoch 5/10
2313/2313 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.6110 - loss: 0.8516 - val_accuracy: 0.6094 - val_loss: 0.8438
Epoch 6/10
2313/2313 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.6121 - loss: 0.8485 - val_accuracy: 0.6119 - val_loss: 0.8378
Epoch 7/10
2313/2313 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.6053 - loss: 0.8552 - val_accuracy: 0.6104 - val_loss: 0.8416
Epoch 8/10
2313/2313 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.6097 - loss: 0.8466 - val_ac

In [214]:
# prompt: simpan model yang telah dibuat
# WARNING:absl:You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`.

joblib.dump(model, "../predictor/nn_model.pkl")

NameError: name 'joblib' is not defined